This time we do something similar, but we have teams.  Each team should perform 2 of the tasks, we still want to minimize the costs


In [11]:
from pulp import *

costs = [[90, 76, 75, 70],
          [35, 85, 55, 65],
          [125, 95, 90, 105],
          [45, 110, 95, 115],
          [60, 105, 80, 75],
          [45, 65, 110, 95]]

team1 = [0, 2, 4]
team2 = [1, 3, 5]
team_max = 2

In [12]:
#Create the problem:

prob = LpProblem("Team schedule Task", LpMinimize)


# First create the variables



num_workers = len(costs)
num_tasks = len(costs[0])

# Create a variable for each worker and each task:
worker_task = [(worker, task) for worker in range(num_workers) for task in range(num_tasks)]


worker_task_vars = LpVariable.dicts("worker_task", worker_task,0, 1, LpInteger)

In [13]:
#As always first add the objective function

prob += lpSum([costs[worker][task] * worker_task_vars[worker, task] for  worker,task in worker_task])

#Each worker can only do one task

for worker in range(num_workers):
    prob += lpSum([worker_task_vars[worker, task] for  task in range(num_tasks)]) <= 1
    
#Each task needs to be done once

for task in range(num_tasks):
    prob += lpSum([worker_task_vars[worker, task] for  worker in range(num_workers)]) == 1

In [14]:
# We add the team constrain
prob += lpSum([worker_task_vars[worker, task] for worker in team1 for task in range(num_tasks)]) <= 2
prob += lpSum([worker_task_vars[worker, task] for worker in team2 for task in range(num_tasks)]) <= 2

# The problem is solved using PuLP's choice of Solver
prob.solve()
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

Status: Optimal
worker_task_(0,_0) = 0.0
worker_task_(0,_1) = 0.0
worker_task_(0,_2) = 1.0
worker_task_(0,_3) = 0.0
worker_task_(1,_0) = 1.0
worker_task_(1,_1) = 0.0
worker_task_(1,_2) = 0.0
worker_task_(1,_3) = 0.0
worker_task_(2,_0) = 0.0
worker_task_(2,_1) = 0.0
worker_task_(2,_2) = 0.0
worker_task_(2,_3) = 0.0
worker_task_(3,_0) = 0.0
worker_task_(3,_1) = 0.0
worker_task_(3,_2) = 0.0
worker_task_(3,_3) = 0.0
worker_task_(4,_0) = 0.0
worker_task_(4,_1) = 0.0
worker_task_(4,_2) = 0.0
worker_task_(4,_3) = 1.0
worker_task_(5,_0) = 0.0
worker_task_(5,_1) = 1.0
worker_task_(5,_2) = 0.0
worker_task_(5,_3) = 0.0


In [15]:
total_cost = 0
#Parse result
for worker_task,v in worker_task_vars.items():
    if v.varValue > 0:
        cost = costs[worker_task[0]][worker_task[1]]
        total_cost += cost
        print("worker {} performs task {} with cost: {} <br>".format(worker_task[0], worker_task[1], cost))
print("Total cost becomes: {}".format(total_cost))
    

worker 0 performs task 2 with cost: 75 <br>
worker 1 performs task 0 with cost: 35 <br>
worker 4 performs task 3 with cost: 75 <br>
worker 5 performs task 1 with cost: 65 <br>
Total cost becomes: 250
